In [2]:
# conda activate ESE527
# Import the library
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import os
from medcat.vocab import Vocab
from medcat.cdb import CDB
from medcat.cat import CAT
from medcat.meta_cat import MetaCAT
import textwrap
from rich import print 
import os
import json
import re
import pickle
import time
from tqdm import tqdm, trange
from concurrent.futures import ThreadPoolExecutor


c:\Users\judyw\Desktop\ESE527\project\.conda\Lib\site-packages\medcat\cat.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


### Prepared data and model


In [3]:
# Load the vocab model we downloaded 
vocab = Vocab.load("medcat_models/vocab.dat")
# Load the cdb model you downloaded
cdb = CDB.load('medcat_models/cdb.dat') 

c:\Users\judyw\Desktop\ESE527\project\.conda\Lib\site-packages\pydantic\main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `set[str]` but got `dict` with value `{}` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
which may or may not work. If you experience any compatibility issues, please reinstall MedCAT
or download the compatible model.


In [4]:
# Run the medcat_status model
mc_status = MetaCAT.load("medcat_models/meta_Status")

c:\Users\judyw\Desktop\ESE527\project\.conda\Lib\site-packages\medcat\meta_cat.py:430: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  meta_cat.model.load_state_dict(torch.loa

In [5]:
# Initial the CAT
# Please run twice
cat = CAT(cdb=cdb, config=cdb.config, vocab=vocab, meta_cats=[mc_status])


Traceback (most recent call last):
  File "c:\Users\judyw\Desktop\ESE527\project\.conda\Lib\site-packages\medcat\pipe.py", line 52, in __init__
    self._nlp = self._init_nlp(config)
                ^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\judyw\Desktop\ESE527\project\.conda\Lib\site-packages\medcat\pipe.py", line 76, in _init_nlp
    return spacy.load(config.general.spacy_model, disable=config.general.spacy_disabled_components)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\judyw\Desktop\ESE527\project\.conda\Lib\site-packages\spacy\__init__.py", line 51, in load
    return util.load_model(
           ^^^^^^^^^^^^^^^^
  File "c:\Users\judyw\Desktop\ESE527\project\.conda\Lib\site-packages\spacy\util.py", line 472, in load_model
    raise IOError(Errors.E050.format(name=name))
OSError: [E050] Can't find model 'spacy_model'. It doesn't seem to be a Python package or a valid path to a data directory.


In [6]:
# Read the MIMIC-III data
# Read all of csv file
# Read the required csv file first
note_event_df = pd.read_csv('Data/NOTEEVENTS.csv', low_memory=False)
patients_df = pd.read_csv('Data/PATIENTS.csv', low_memory=False)
admissions_df = pd.read_csv('Data/ADMISSIONS.csv',  low_memory=False)


In [7]:
# Just a look of the the noteevent 
note_event_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


In [8]:
# Load the doc2text
"""
ROW ID as the key
Notes as the item
Target: Reduce the read the note.csv several times. We will
use the row id as the index to connect the other info
"""

# Convert the Noteevents to the pickle files
note_required_text = note_event_df.set_index('ROW_ID')['TEXT'].to_dict()



In [9]:
# Save the dictionary into the pickle file doc2text
with open('doc2text.pickle', 'wb') as f:
    pickle.dump(note_required_text, f)

size_note = len(note_required_text)
# Check the size of the noteevents and the pickle file already generated
print(f"We already the doc2text file and we have {size_note} clinical notes.")

We already the doc2text file and we have 2083180 clinical notes.

In [10]:
# If we have a doc2text right now, we need to load it
with open('doc2text.pickle', 'rb') as f:
    doc2text = pickle.load(f)
print("Opened doc2text pickle file")

Opened doc2text pickle file

In [11]:
"""
construct the doc2info dictionary, the key should be the row id and the 
chartdate and subject id should be value
"""
doc2info = {}
info_df = note_event_df[['ROW_ID', 'SUBJECT_ID', 'CHARTDATE']]
doc2info = info_df.set_index('ROW_ID').to_dict(orient='index')

# Check for the first 3 dict
for doc_id, info in list(doc2info.items())[:3]:
    print(doc_id, "=>", info)


174 =>
{'SUBJECT_ID': 22532, 'CHARTDATE': '2151-08-04'}

175 =>
{'SUBJECT_ID': 13702, 'CHARTDATE': '2118-06-14'}

176 =>
{'SUBJECT_ID': 13702, 'CHARTDATE': '2119-05-25'}

In [12]:
# Data cleaning method 
def data_clean(text):
    # Remove this format
    # Remove the token like this "[**Name**]"
    clean_step1 = re.sub(r'\[\*\*.*?\*\*\]', ' ', text)
    cleaned = re.sub(r'\s+', ' ', clean_step1)
    final_cleaned = cleaned.strip()
    return final_cleaned


In [13]:
# Call the clean function
for id, text in doc2text.items():
    doc2text[id] = data_clean(text)



In [14]:
# Small check for the cleaned data
for doc_id, text in list(doc2text.items())[:1]:
    print(doc_id, "=>", text)

174 => Admission Date: Discharge Date: Service: ADDENDUM: RADIOLOGIC STUDIES: Radiologic studies also included a 
chest CT, which confirmed cavitary lesions in the left lung apex consistent with infectious process/tuberculosis. 
This also moderate-sized left pleural effusion. HEAD CT: Head CT showed no intracranial hemorrhage or mass effect, 
but old infarction consistent with past medical history. ABDOMINAL CT: Abdominal CT showed lesions of T10 and 
sacrum most likely secondary to osteoporosis. These can be followed by repeat imaging as an outpatient. , M.D. 
Dictated By: MEDQUIST36 D: 12:11 T: 12:21 JOB#:

In [15]:
def extract_medcat_entity(text):
    entities_info = []
    doc = cat.get_entities(text)
    for row_id, entity in doc['entities'].items():
        if not isinstance(entity, dict):
            continue
        try:
            cui = entity.get('cui')
            name = entity.get('pretty_name')
            sem_type = entity.get('types',[])
            print(sem_type)
            start = entity.get('start')
            end = entity.get('end')
            if 'meta_anns' in entity:
                status_info = entity['meta_anns'].get('Status')  # or whatever your meta model's name is
            if status_info:
                status = status_info.get('value')  # e.g. "Affirmed" vs "Negated"
            # Find the current content (+50 tokens)
            sent_start = text.rfind('.', 0, start) 
            sent_end = text.find('.', end)   
            if sent_start == -1:
                sent_start = max(0, start - 50) # Paper mentioned
            else:
                sent_start += 1 
            if sent_end == -1:
                sent_end = min(len(text), end + 50)
            context_sentence = text[sent_start:sent_end].strip()

            entities_info.append({
                'cui': cui,
                'name': name,
                'type': sem_type,
                'start': start,
                'end': end,
                'context': context_sentence
            })
        except Exception as e:
            print(f"Error processing entity: {entity}")
            continue
    return entities_info



In [ ]:
# Target Type 
target_types = {'disorders','symptoms', 'findings', 'medications'}

# We should replace the medicak concept with the cui 
def replace_text_cui(text, entities):
    # Sort thhe enitities through the start index which may help us to avoid modify the position of 
    entities = sorted(entities, key = lambda e: e['start'])
    result_parts = []
    last_index = 0
    for ent in entities:
        # Skip the type we do not expect
        if ent['type'] not in target_types:
            continue
        # Append the text from the last index to next enitity
        result_parts.append(text[last_index : ent['start']])
        # Append the CUI code in place of the concept mention
        result_parts.append(ent['cui'])
        # Update the last index to the end of the replaced entity
        last_idx = ent['end']
    # Append any remaining text after the last entity
    result_parts.append(text[last_idx:])
    # Join all parts into the final text
    return "".join(result_parts)